## XBRL US API - Python example  
This notebook contains example Python code to use the XBRL US API (https://xbrl.us/home/use/xbrl-api/)    
  
**Made by:** [Ties de Kok](https://www.tiesdekok.com) and [Beth Blankespoor](https://foster.uw.edu/faculty-research/directory/elizabeth-blankespoor/)

## Imports

In [1]:
import os, re, sys, json
import requests

In [2]:
import pandas as pd
import numpy as np

In [3]:
import getpass

## Api documentation

https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails

## Generate an access token
The access token will expire after 60 minutes.

**You have two options:**

**Option 1:** obtain your own credentials

1. Obtain credentials here: https://xbrl.us/home/use/xbrl-api-community/#provisioning
2. Update 'login_cred.json' with your `client_id`, `client_secret`, and `username`  
3. Set `USE_TEMP_CRED` to `False`
4. Input your password when asked

**Option 2:** Use my temporary credentials for demo purposes only

1. Set `USE_TEMP_CRED` to `True`
2. Input your email address when asked


In [4]:
USE_TEMP_CRED = True
if USE_TEMP_CRED:
    user_email = input(prompt="Please type your email address here: ")
    access_token = requests.get('https://tdekok-xbrlapi.builtwithdark.com/gettoken?platform=aaa-{}'.format(user_email)).text.replace('"', "")
else:
    endpoint = 'https://api.xbrl.us'
    endpoint_auth = endpoint + '/oauth2/token'

    with open('login_cred.json', 'r') as f:
        login_cred = json.loads(f.read())
    password = getpass.getpass(prompt = 'Password: ')
    body_auth = {'grant_type' : 'password', 
                'client_id' : login_cred['client_id'], 
                'client_secret' : login_cred['client_secret'], 
                'username' : login_cred['username'] ,
                'password' : password,
                'platform' : 'pc'}
    res = requests.post(endpoint_auth, data=body_auth)
    auth_json = res.json()
    access_token = auth_json['access_token']

Please type your email address here: your@email.com


## Make a query

### Define the fields you would like returned

In [5]:
fields = ['entity.cik',
         'entity.name.sort(ASC)',
         'dts.id',
         'fact.id',
         'report.filing-date',
         'period.fiscal-year',
         'period.instant',
         'report.document-type',
         'concept.local-name',
         'dimensions.count',
         'dimension.local-name.sort(ASC)',
         'member.local-name',
         'fact.value',
         'unit',
         'fact.decimals',
          'fact.ultimus-index',
         'dimension.namespace',
         'member.namespace',
          'fact.has-dimensions'
         ]

### Define the XBRL elements (tags) you'd like 

In [6]:
XBRL_Elements = [
     'IntangibleAssetsNetExcludingGoodwill',
     'FiniteLivedIntangibleAssetsNet',
     'IndefiniteLivedIntangibleAssetsExcludingGoodwill'
                ]

### Define the companies you'd like

In [7]:
companies_cik = ['0000789019', ## Microsoft (MSFT)
                 '0001018724', ## Amazon (AMZN)
                 '0001652044', ## Alphabet (GOOG)
                 '0001326801', ## Facebook (FB)
                 '0001065280', ## Netflix (NFLX)
                 '0001166691', ## Comcast (CMCSA)
                 '0001326380', ## GameStop (GME)
                 '0000732717', ## AT&T (T)
                ]

### Define the years you'd like

In [8]:
years = ['2019'] ## Use commas between for multiple years, e.g., '2018','2019'
#years = [str(2013 + i) for i in range(8)] ## Years 2013 to 2020

### Specify if you want dimensions, no dimension, or all values

In [9]:
has_dimensions = 'FALSE'  ## TRUE for require dimensions, FALSE for no dimensions, ALL for all values

### Specify the report types that you want

In [10]:
report_types = ['10-K', '10-K/A']

## Execute query

In [11]:
search_endpoint = 'https://api.xbrl.us/api/v1/fact/search'

In [12]:
params = {'concept.local-name': ','.join(XBRL_Elements),
     'period.fiscal-period': 'Y',
     'period.fiscal-year': ','.join(years),
     'unit': 'USD',
     'entity.cik': ','.join(companies_cik),
     'report.document-type': ','.join(report_types),
     }  
if has_dimensions == 'ALL':
    dimension_options = ['TRUE', 'FALSE']
else:
    dimension_options = [has_dimensions]
    
all_res_list = []
for dimensions_param in dimension_options:
    print('Getting the data for: "fact.has-dimensions" = {}'.format(dimensions_param))
    ### Every request will return a max of 2000 results. So we loop until all results are retrieved. 
    done_retrieving_all_results = False
    offset = 0
    while not done_retrieving_all_results:
        params['fact.has-dimensions'] = dimensions_param
        params['fields'] = ','.join(fields) + ',fact.offset({})'.format(offset) 
        res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
        
        ## Interpret as JSON
        res_json = res.json()
            
        ## Get the results
        ### Retrieve the data list
        res_list = res_json['data']
        
        ### Add to the results
        all_res_list += res_list
        
        ## Pagination check
        paging_dict = res_json['paging']
        print('Number of records retrieved: ', paging_dict['count'])
        if paging_dict['count'] >= 2000:
            offset += paging_dict['count']
        else:
            done_retrieving_all_results = True
    
## Convert to a DataFrame
res_df = pd.DataFrame(all_res_list)
print('\nNumber of results that meet the criteria: {}'.format(len(res_df)))

Getting the data for: "fact.has-dimensions" = FALSE
Number of records retrieved:  13

Number of results that meet the criteria: 13


## Show results

In [13]:
columns_to_hide = ['entity.cik', 'fact.id', 'fact.decimals', 'dimension.namespace', 'member.namespace']
columns_to_show = [column for column in res_df.columns if column not in columns_to_hide]

res_df.sort_values(by=['entity.name','dts.id','concept.local-name','fact.ultimus-index','dimension.local-name'], inplace=True)
res_df[columns_to_show].head(20)

,concept.local-name,dimension.local-name,dimensions.count,dts.id,entity.name,fact.has-dimensions,fact.ultimus-index,fact.value,member.local-name,period.fiscal-year,period.instant,report.document-type,report.filing-date,unit
3,FiniteLivedIntangibleAssetsNet,,0,362382,"AMAZON.COM, INC.",False,1,4021000000,,2019,2020-01-01,10-K,2020-01-31,USD
4,IntangibleAssetsNetExcludingGoodwill,,0,362382,"AMAZON.COM, INC.",False,1,4049000000,,2019,2020-01-01,10-K,2020-01-31,USD
5,IndefiniteLivedIntangibleAssetsExcludingGoodwill,,0,365998,AT&T INC.,False,1,101392000000,,2019,2020-01-01,10-K,2020-02-20,USD
2,FiniteLivedIntangibleAssetsNet,,0,362758,Alphabet Inc.,False,1,1979000000,,2019,2020-01-01,10-K,2020-02-04,USD
1,IntangibleAssetsNetExcludingGoodwill,,0,362758,Alphabet Inc.,False,7,1979000000,,2019,2020-01-01,10-K,2020-02-04,USD
0,IntangibleAssetsNetExcludingGoodwill,,0,362758,Alphabet Inc.,False,8,1979000000,,2019,2020-01-01,10-K,2020-02-04,USD
6,FiniteLivedIntangibleAssetsNet,,0,362131,"Facebook, Inc.",False,4,894000000,,2019,2020-01-01,10-K,2020-01-30,USD
8,FiniteLivedIntangibleAssetsNet,,0,362131,"Facebook, Inc.",False,5,894000000,,2019,2020-01-01,10-K,2020-01-30,USD
7,IntangibleAssetsNetExcludingGoodwill,,0,362131,"Facebook, Inc.",False,4,894000000,,2019,2020-01-01,10-K,2020-01-30,USD
9,IntangibleAssetsNetExcludingGoodwill,,0,322837,GameStop Corp.,False,1,33500000,,2019,2019-02-03,10-K,2019-04-02,USD


## Show dimensions example (if exists)

In [14]:
res_df[res_df['dimensions.count'] > 1].sort_values(by=['entity.name','dts.id','concept.local-name','fact.ultimus-index',
                'dimension.local-name'])[columns_to_show].head(20)

,concept.local-name,dimension.local-name,dimensions.count,dts.id,entity.name,fact.has-dimensions,fact.ultimus-index,fact.value,member.local-name,period.fiscal-year,period.instant,report.document-type,report.filing-date,unit
